In [1]:
import os
import copy
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mlp
%matplotlib inline
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM, BartForConditionalGeneration,BartTokenizer
import shap
import scipy as sp
import nlp
import torch
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
pd.set_option("max_rows", None)
from shap.utils import assert_import, record_import_error, safe_isinstance, make_masks, OpChain, MaskedModel
from shap import maskers, links
from transformers.file_utils import ModelOutput
#from shap.utils import cal_conditional_logits
import sys
np.set_printoptions(formatter={'float': lambda x: "{0:0.5f}".format(x)})

In [2]:
tokenizer = BartTokenizer.from_pretrained("sshleifer/distilbart-xsum-12-6")
model =  BartForConditionalGeneration.from_pretrained("sshleifer/distilbart-xsum-12-6").cuda()

In [2]:
sent1="Hi my dog is cute"
sent2="Hello my puppy is cute"
#input_ids = torch.tensor(tokenizer.encode(sent1)).unsqueeze(0).cuda()
#output_ids = torch.tensor(tokenizer.encode(sent2)).unsqueeze(0).cuda()

In [4]:
input_ids

tensor([[    0, 30086,   127,  2335,    16, 11962,     2]], device='cuda:0')

In [5]:
output_ids

tensor([[    0, 31414,   127, 20830,    16, 11962,     2]], device='cuda:0')

In [6]:
model.eval()
#model.train()
with torch.no_grad():
    outputs = model(input_ids=input_ids, decoder_input_ids=output_ids,labels=output_ids,return_dict=True)

In [7]:
outputs.logits.shape

torch.Size([1, 7, 50264])

In [8]:
lm_tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
lm_model = AutoModelWithLMHead.from_pretrained("distilgpt2").cuda()

In [9]:
lm_input_ids = torch.tensor(lm_tokenizer.encode(sent1)).unsqueeze(0).cuda()
lm_output_ids = torch.tensor(lm_tokenizer.encode(sent2)).unsqueeze(0).cuda()

In [10]:
lm_input_ids

tensor([[17250,   616,  3290,   318, 13779]], device='cuda:0')

In [11]:
lm_output_ids

tensor([[15496,   616, 26188,   318, 13779]], device='cuda:0')

In [12]:
concat_inputs = torch.cat((lm_input_ids,lm_output_ids),dim=-1)

In [13]:
concat_inputs

tensor([[17250,   616,  3290,   318, 13779, 15496,   616, 26188,   318, 13779]],
       device='cuda:0')

In [14]:
past=None
with torch.no_grad():
    output = lm_model(lm_input_ids)

In [15]:
lm_output_ids[0,0].unsqueeze(0)

tensor([15496], device='cuda:0')

In [16]:
past=None
for i in range(0,lm_output_ids.shape[1]):
    if i==0:
        with torch.no_grad():
            output, past = lm_model(lm_input_ids, past=past)           
    else:
        with torch.no_grad():
            output, past = lm_model(lm_output_ids[0,i-1].unsqueeze(0), past=past)
    next_token_logits = output[..., -1, :].detach().cpu()
    print(next_token_logits)

tensor([[-48.9168, -52.4876, -55.5191,  ..., -65.9269, -62.6038, -51.5396]])
tensor([-42.9091, -48.0814, -51.4108,  ..., -57.4924, -56.4301, -48.2307])
tensor([-39.2923, -41.9795, -44.2819,  ..., -49.5130, -47.5725, -42.0874])
tensor([-43.7650, -47.3087, -49.6883,  ..., -59.5156, -54.8545, -46.3371])
tensor([-47.3736, -50.3058, -51.4557,  ..., -61.9204, -56.3025, -50.7423])


In [17]:
lm_model.eval()
past=None
with torch.no_grad():
    outputs = lm_model(concat_inputs,return_dict=True)

In [18]:
outputs.logits

tensor([[[-32.9849, -31.3989, -33.2205,  ..., -44.7180, -44.6273, -33.4917],
         [-50.4431, -50.5061, -51.8230,  ..., -53.8949, -55.2346, -51.0417],
         [-56.9040, -59.4501, -61.7221,  ..., -70.2842, -65.0359, -59.3530],
         ...,
         [-43.7650, -47.3087, -49.6883,  ..., -59.5156, -54.8545, -46.3371],
         [-47.3736, -50.3058, -51.4557,  ..., -61.9204, -56.3025, -50.7422],
         [-38.3973, -41.4092, -42.6722,  ..., -54.3608, -51.5736, -39.0006]]],
       device='cuda:0')

In [19]:
for i in range(lm_input_ids.shape[1]-1,lm_input_ids.shape[1]+lm_output_ids.shape[1]-1):
    print(outputs.logits[:,i,:])

tensor([[-48.9168, -52.4876, -55.5191,  ..., -65.9269, -62.6038, -51.5396]],
       device='cuda:0')
tensor([[-42.9091, -48.0814, -51.4108,  ..., -57.4924, -56.4301, -48.2307]],
       device='cuda:0')
tensor([[-39.2923, -41.9795, -44.2819,  ..., -49.5130, -47.5725, -42.0874]],
       device='cuda:0')
tensor([[-43.7650, -47.3087, -49.6883,  ..., -59.5156, -54.8545, -46.3371]],
       device='cuda:0')
tensor([[-47.3736, -50.3058, -51.4557,  ..., -61.9204, -56.3025, -50.7422]],
       device='cuda:0')


In [20]:
for i in range(lm_input_ids.shape[1]-1,lm_input_ids.shape[1]+lm_output_ids.shape[1]):
    print(outputs.logits[:,i,:])

tensor([[-48.9168, -52.4876, -55.5191,  ..., -65.9269, -62.6038, -51.5396]],
       device='cuda:0')
tensor([[-42.9091, -48.0814, -51.4108,  ..., -57.4924, -56.4301, -48.2307]],
       device='cuda:0')
tensor([[-39.2923, -41.9795, -44.2819,  ..., -49.5130, -47.5725, -42.0874]],
       device='cuda:0')
tensor([[-43.7650, -47.3087, -49.6883,  ..., -59.5156, -54.8545, -46.3371]],
       device='cuda:0')
tensor([[-47.3736, -50.3058, -51.4557,  ..., -61.9204, -56.3025, -50.7422]],
       device='cuda:0')
tensor([[-38.3973, -41.4092, -42.6722,  ..., -54.3608, -51.5736, -39.0006]],
       device='cuda:0')


In [21]:
lm_input_ids.shape[1]

5

In [22]:
lm_output_ids.shape

torch.Size([1, 5])

In [3]:
class GenerateLogits:
    def __init__(self,model="distilgpt2",tokenizer=None):
        if isinstance(model,str):
            # load model and tokenizer from transformers library
            self.model = AutoModelWithLMHead.from_pretrained(model)
            self.tokenizer = AutoTokenizer.from_pretrained(model)
        else:
            self.model = model
            self.tokenizer = tokenizer
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.model = self.model.to(self.device)
        null_tokens = self.tokenizer.encode("")

        if len(null_tokens) == 0:
            self.keep_prefix = 0
            self.keep_suffix = 0
        elif len(null_tokens) == 1:
            null_token = null_tokens[0]
            assert (('eos_token' in self.tokenizer.special_tokens_map) or ('bos_token' in self.tokenizer.special_tokens_map)), "No eos token or bos token found in tokenizer!"
            if ('eos_token' in self.tokenizer.special_tokens_map) and (self.tokenizer.decode(null_token) == self.tokenizer.special_tokens_map['eos_token']):
                self.keep_prefix = 0
                self.keep_suffix = 1
            elif ('bos_token' in self.tokenizer.special_tokens_map) and (self.tokenizer.decode(null_token) == self.tokenizer.special_tokens_map['bos_token']):
                self.keep_prefix = 1
                self.keep_suffix = 0
        else:
            assert len(null_tokens) % 2 == 0, "An odd number of boundary tokens are added to the null string!"
            self.keep_prefix = len(null_tokens) // 2
            self.keep_suffix = len(null_tokens) // 2

    def get_teacher_forced_logits(self,source_sentence_ids,target_sentence_ids):
        self.model.eval()
        if self.model.config.is_encoder_decoder:
            # assigning decoder start token id 
            decoder_start_token_id = None

            if hasattr(self.model.config, "decoder_start_token_id") and self.model.config.decoder_start_token_id is not None:
                decoder_start_token_id = self.model.config.decoder_start_token_id
            elif hasattr(self.model.config, "bos_token_id") and self.model.config.bos_token_id is not None:
                decoder_start_token_id = self.model.config.bos_token_id
            elif (hasattr(self.model.config, "decoder") and hasattr(self.model.config.decoder, "bos_token_id") and self.model.config.decoder.bos_token_id is not None):
                decoder_start_token_id = self.model.config.decoder.bos_token_id
            else:
                raise ValueError(
                    "No decoder_start_token_id or bos_token_id defined in config for encoder-decoder generation"
                )

            target_sentence_start_id = torch.tensor([[decoder_start_token_id]]).to(self.device)
            target_sentence_ids = torch.cat((target_sentence_start_id,target_sentence_ids),dim=-1)
            with torch.no_grad():
                outputs = self.model(input_ids=source_sentence_ids, decoder_input_ids=target_sentence_ids, labels=target_sentence_ids, return_dict=True)
            logits=outputs.logits.detach().cpu().numpy()
        else:
            combined_sentence_ids = torch.cat((source_sentence_ids,target_sentence_ids),dim=-1)
            with torch.no_grad():
                outputs = self.model(input_ids=combined_sentence_ids, return_dict=True)
            logits=outputs.logits.detach().cpu().numpy()[:,source_sentence_ids.shape[1]-1:,:]
        del outputs
        return logits

    def get_sentence_ids(self, source_sentence,target_sentence):
        source_sentence_ids = torch.tensor([self.tokenizer.encode(source_sentence)])
        if self.keep_suffix > 0:
            target_sentence_ids = torch.tensor([self.tokenizer.encode(target_sentence)])[:,self.keep_prefix:-self.keep_suffix]
        else:
            target_sentence_ids = torch.tensor([self.tokenizer.encode(target_sentence)])[:,self.keep_prefix:]
        return source_sentence_ids.to(self.device), target_sentence_ids.to(self.device)

    def get_output_names(self, sentence):
        return self.tokenizer.tokenize(sentence)

    def generate(self,source_sentence, target_sentence):
        source_sentence_ids, target_sentence_ids = self.get_sentence_ids(source_sentence,target_sentence)
        logits = self.get_teacher_forced_logits(source_sentence_ids,target_sentence_ids)
        conditional_logits = []
        for i in range(0,logits.shape[1]-1):
            probs = (np.exp(logits[0][i]).T / np.exp(logits[0][i]).sum(-1)).T
            logit_dist = sp.special.logit(probs)
            conditional_logits.append(logit_dist[target_sentence_ids[0,i].item()])
        return np.array(conditional_logits)

In [8]:
logit_model = GenerateLogits()

In [9]:
print(logit_model.generate(source_sentence=sent1,target_sentence=sent2))

[-12.81402 -3.68543 -4.37839 -0.50151 -0.62419]


In [13]:
logit_model = GenerateLogits(model = "sshleifer/distilbart-xsum-12-6")

In [14]:
print(logit_model.generate(source_sentence=sent1,target_sentence=sent2))

[-5.35224 -4.15271 -6.16725 -1.70498 0.41188]


In [15]:
logit_model.tokenizer.tokenize("Its a sunnny day")

['Its', 'Ġa', 'Ġsun', 'n', 'ny', 'Ġday']

In [16]:
logit_model.tokenizer.encode("Its a sunnny day")

[0, 30872, 10, 3778, 282, 2855, 183, 2]